In [ ]:
!pip install -q transformers sentence-transformers faiss-cpu gradio joblib torch scikit-learn datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 42.2 MB/s eta 0:00:00


In [ ]:
import torch
import joblib
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_DIR = "/content/drive/MyDrive/NLP_Project/model"

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
label_encoder = joblib.load(f"{MODEL_DIR}/label_encoder.pkl")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

print("✅ BioClinicalBERT loaded")


✅ BioClinicalBERT loaded


In [ ]:

from datasets import load_dataset
import pandas as pd

dataset = load_dataset("fzkuji/HealthCareMagic-100k", split="train")
hc_df = dataset.to_pandas()

hc_df = hc_df.dropna(subset=["instruction", "output"])
hc_df["text"] = hc_df["instruction"] + " " + hc_df["output"]
hc_df = hc_df[hc_df["text"].str.len() > 50]

print("Total samples:", len(hc_df))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/287 [00:00<?, ?B/s]

HealthCareMagic-100k.json:   0%|          | 0.00/144M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/112165 [00:00<?, ? examples/s]

Total samples: 112165


In [ ]:
pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 31.6 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Sample for Colab safety
hc_sample = hc_df.sample(20000, random_state=42).reset_index(drop=True)

embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

embeddings = embedder.encode(
    hc_sample["text"].tolist(),
    convert_to_numpy=True,
    show_progress_bar=True
)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print("✅ HealthCareMagic RAG index ready")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

✅ HealthCareMagic RAG index ready


In [ ]:
def predict_top3(symptoms):
    inputs = tokenizer(
        symptoms,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=64
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)[0]

    top3 = torch.topk(probs, k=3)

    results = []
    for idx, score in zip(top3.indices, top3.values):
        disease = label_encoder.inverse_transform([idx.item()])[0]
        results.append((disease, float(score)))

    return results


In [ ]:
def retrieve_hcm_context(query, k=2):
    query_embedding = embedder.encode(query, convert_to_numpy=True).reshape(1, -1)
    _, indices = index.search(query_embedding, k)

    answers = []
    for idx in indices[0]:
        answers.append(hc_sample.iloc[idx]["output"])

    return "\n\n".join(answers)


In [ ]:
followup_questions = {
    "fever": [
        "How long have you had the fever?",
        "Is the fever high (>38.5°C)?",
        "Do you experience chills or sweating?"
    ],
    "cough": [
        "Is the cough dry or productive?",
        "Do you have shortness of breath or chest pain?"
    ],
    "headache": [
        "Is the headache severe?",
        "Do you feel nausea or sensitivity to light?"
    ],
    "pain": [
        "Where exactly is the pain located?",
        "Is the pain constant or intermittent?"
    ],
    "urination": [
        "Do you feel burning or pain during urination?",
        "Is urination more frequent than usual?"
    ]
}


In [ ]:
def extract_keywords(text):
    return [k for k in followup_questions if k in text.lower()]


In [ ]:
chat_state = {
    "symptoms": "",
    "followups_asked": False
}


In [ ]:
def interactive_chatbot(user_input):
    # First turn: ask follow-up questions
    if not chat_state["followups_asked"]:
        chat_state["symptoms"] = user_input

        keywords = extract_keywords(user_input)
        questions = []

        for key in keywords:
            questions.extend(followup_questions[key])

        questions = questions[:2]  # avoid overload

        if questions:
            chat_state["followups_asked"] = True
            q_text = "\n".join([f"- {q}" for q in questions])

            return (
                "🩺 To give a more precise answer, I need a bit more information:\n\n"
                f"{q_text}"
            )

    # Second turn: final prediction + RAG
    full_symptoms = chat_state["symptoms"] + " " + user_input

    predictions = predict_top3(full_symptoms)
    rag_context = retrieve_hcm_context(full_symptoms)

    response = "🩺 **Top-3 Possible Conditions:**\n"
    for i, (disease, conf) in enumerate(predictions, 1):
        response += f"{i}. **{disease}** — {conf:.2f}\n"

    response += "\n📚 **Medical information (from real doctor Q&A):**\n"
    response += rag_context
    response += "\n\n⚠️ *Educational use only. Not a medical diagnosis.*"

    # Reset state
    chat_state["symptoms"] = ""
    chat_state["followups_asked"] = False

    return response


In [ ]:
import gradio as gr

with gr.Blocks(title="Interactive Medical Chatbot") as demo:
    gr.Markdown("## 🩺 Interactive AI Medical Assistant")
    gr.Markdown(
        "The assistant asks follow-up questions to improve prediction accuracy."
    )

    chatbot = gr.Chatbot()
    state = gr.State([])

    user_input = gr.Textbox(
        label="Your message",
        placeholder="Describe your symptoms...",
        lines=3
    )

    send = gr.Button("Send")

    def respond(message, history):
        reply = interactive_chatbot(message)
        history.append((message, reply))
        return history, history

    send.click(respond, inputs=[user_input, state], outputs=[chatbot, state])

demo.launch()


/tmp/ipython-input-2132628745.py:9: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()
/tmp/ipython-input-2132628745.py:9: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0358131efc5448ef33.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
